In [ ]:
from electroviz.core.experiment import Experiment
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.utils.cross_corr import plot_cross_corr
import matplotlib.pyplot as plt

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("depth")

for ctx_idx, ctx_unit in enumerate(pop0_sort[20:300]):
    for lgn_idx, lgn_unit in enumerate(pop0_sort[590:730]):
        file_path = "G:/Shared drives/Neuropixels/Figures/RG001/CrossCorr/"
        file_name = "Probe1_Unit{0}_Probe1_Unit{1}.png".format(str(ctx_idx+20), str(lgn_idx+590))
        plot_cross_corr((ctx_unit, lgn_unit), save_path=file_path+file_name)
        plt.close("all")

%reset -f

from electroviz.core.experiment import Experiment
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.utils.cross_corr import plot_cross_corr
import matplotlib.pyplot as plt

exp = Experiment("E:/RG008/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("depth")

pop1 = exp.populations[1]
pop1 = pop1.remove(pop1.units["total_spikes"] < 100)
pop1_sort = pop1.sort("depth")

for ctx_idx, ctx_unit in enumerate(pop1_sort[:130]):
    for lgn_idx, lgn_unit in enumerate(pop0_sort[230:480]):
        file_path = "G:/Shared drives/Neuropixels/Figures/RG008/CrossCorr/"
        file_name = "Probe2_Unit{0}_Probe0_Unit{1}.png".format(str(ctx_idx), str(lgn_idx+230))
        plot_cross_corr((ctx_unit, lgn_unit), save_path=file_path+file_name)
        plt.close("all")

In [1]:
from electroviz.core.experiment import Experiment
from electroviz.core.kernel import SparseNoiseKernel
from electroviz.core.kernel import StaticGratingsKernel
from electroviz.utils.cross_corr import cross_corr

exp = Experiment("E:/RG001/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 100)
pop0_sort = pop0.sort("depth")

ctx_unit = pop0_sort[92]
lgn_unit = pop0_sort[657]

xcorr_raw, xcorr_filt = cross_corr((ctx_unit, lgn_unit))